In [1]:
# Description : This notebook is focused on building an artificial recurrent neural network calles LSTM 
# (Long Short Term Memory) to predict the closing price of a corporation using the past 60 days data. 

In [2]:
import math
import pandas as pd
import numpy as np
import pandas_datareader as web
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from datetime import date
import yfinance as yf

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [3]:
today = date.today().strftime("%Y-%m-%d")
df = yf.download("MMM",'2021-08-07',today)

Exception in thread Thread-4:
Traceback (most recent call last):
  File "D:\programming language\python\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "D:\programming language\python\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "D:\programming language\python\lib\site-packages\multitasking\__init__.py", line 102, in _run_via_pool
    return callee(*args, **kwargs)
  File "D:\programming language\python\lib\site-packages\yfinance\multi.py", line 167, in _download_one_threaded
    data = _download_one(ticker, start, end, auto_adjust, back_adjust,
  File "D:\programming language\python\lib\site-packages\yfinance\multi.py", line 179, in _download_one
    return Ticker(ticker).history(period=period, interval=interval,
  File "D:\programming language\python\lib\site-packages\yfinance\base.py", line 157, in history
    data = data.json()
  File "D:\programming language\python\lib\site-packages\requests\models.py", line 900, i

KeyboardInterrupt: 

In [ ]:
print(f"Shape : {df.shape}")
print(df.tail())
print(df.info())

In [ ]:
plt.figure(figsize=(16,8))
plt.title('Close Price History')
plt.plot(df['Close'])
plt.xlabel('Date')
plt.ylabel('Close Price USD($)',fontsize=18)
plt.show()

In [ ]:
data = df.filter(['Close'])
dataset = data.values
training_data_len = math.ceil(len(dataset) * 0.8)
training_data_len

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)
print(f"Raw Data : \n{dataset}")
print(f"Scaled Data : \n{scaled_data}")

In [ ]:
train_data = scaled_data[0:training_data_len,:]
x_train = []
y_train = []
for i in range(60,len(train_data)):
    x_train.append(train_data[i-60:i,0])
    y_train.append(train_data[i,0])

x_train, y_train = np.array(x_train), np.array(y_train)

In [ ]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_train.shape

In [ ]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

In [ ]:
model.compile(optimizer='adam',loss='mean_squared_error')

In [ ]:
model.fit(x_train,y_train,batch_size=1,epochs=1)

In [ ]:
test_data = scaled_data[training_data_len-60:,:]
x_test = []
y_test = dataset[training_data_len:,:]
for i in range(60,len(test_data)):
    x_test.append(test_data[i-60:i,0])

In [ ]:
x_test = np.array(x_test)

In [ ]:
x_test

In [ ]:
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [ ]:
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

In [ ]:
rmse = np.sqrt(np.mean(predictions - y_test)**2)
rmse

In [ ]:
train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions
plt.figure(figsize=(16,8))
plt.title('Model')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.plot(train['Close'])
plt.plot(valid[['Close','Predictions']])
plt.legend(['Train','Value','Predictions'],loc='lower right')
plt.show()

In [ ]:
valid

In [ ]:
apple_quote = df = yf.download('MMM','2010-01-01','2021-05-29')
new_df = apple_quote.filter(['Close'])
last_60_days = new_df[-60:].values
last_60_days_scaled = scaler.transform(last_60_days)
X_test = []
X_test.append(last_60_days_scaled)
X_test = np.array(X_test)
X_test = np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
pred_price = model.predict(X_test)
pred_price = scaler.inverse_transform(pred_price)
pred_price

In [ ]:
apple_quote_2 = yf.download('MMM','2021-05-30','2021-05-30')
apple_quote_2

In [ ]:
apple_quote